# Homework 1 - Part C

Your name:

Email (one used on the moodle): 

In [ ]:
import numpy as np
from numpy.linalg import norm
from matplotlib import pyplot as plt

## Optimization problem

We consider the ordinary least-squares problem (studied in the course) with the following notations:
\begin{eqnarray*}
A\in \mathbb{R}^{n\times d}, \quad x^* = 1\in \mathbb{R}^d, \quad b = Ax^*\\
f(x) =\frac{1}{2}\| Ax-b\|^2.
\end{eqnarray*}
Least-squares estimation amounts to finding a minimizer of $f$: $f^* =\min_x f(x) =0$.

In the course, we analyzed the gradient descent algorithm. Here we have:
\begin{eqnarray*}
\nabla f(x) = A^T(Ax-b),
\end{eqnarray*}
so that we can write the algorithm as:
\begin{eqnarray*}
x_0 &=& 0\\
x_{k+1} &=& x_k-\gamma A^T(Ax_k-b).
\end{eqnarray*}

In [ ]:
rng   = np.random.default_rng(1)
n   = 55
d   = 50
A   = rng.normal( size=(n,d) )
xstar = np.ones( (d,1) )
b   = A@xstar

f   = lambda x : norm(A@x-b)**2/2
grad= lambda x : A.T@( A@x-b )

fstar   = 0
x0      = np.zeros((d,1))

## Gradient descent

Below is the corresponding code:

In [ ]:
def gradientDescent(f,grad,stepsize,x0,maxiter=1e3):
    x = x0.copy()
    xhist = []
    for k in range(int(maxiter)):
        x -= stepsize*grad(x)
        xhist.append( norm(x-xstar) )
    return x, xhist

In the course, we showed the closed form expression:
\begin{eqnarray*}
x_{k+1} - x^* &=& x_k-x^* -\gamma A^T A(x_k-x^*)\\
x_{k+1} - x^* &=& \left( I- \gamma A^TA \right)(x_k-x^*)\\
&=& \left( I- \gamma H \right)^k(x_0-x^*).
\end{eqnarray*}
The Hessian matrix $H = A^TA$ play a crucial role. $H$ belongs to the set of $d\times d$ symmetric matrices $\mathcal{S}_d$. We define $\mu=\lambda_{\min}(H)$ and $L=\lambda_\max(H)$ so that $\kappa=\frac{L}{\mu}$ is the condition number.

In [ ]:
evals = np.linalg.eigvals(A.T@A)
L     = np.max(evals)
mu    = np.min(evals)
kappa = L/mu
print(f"L is {L:.2f}, mu is {mu:.2f}, condition number is {kappa:.2e}")

In order to maximize the speed of the algorithm, we choose $\gamma$ to minimize the eigenvalues of $I- \gamma A^TA$, i.e. we minimize $\max_{\lambda\in[\mu,L]}|1-\gamma \lambda|$. We showed that the optimal value for $\gamma$ is $\gamma = \frac{2}{L+\mu}$. Another possible weaker choice is $\gamma =\frac{1}{L}$ ensuring that the maximum eigenvalue is still strictly less than one.

In [ ]:
maxiter = 1e4
x_gd, xhist_gd  = gradientDescent(f,grad,1/L,x0,maxiter=maxiter)

x_gdopti, xhist_gdopti  = gradientDescent(f,grad,2/(L+mu),x0,maxiter=maxiter)

In [ ]:
plt.figure(figsize=(12,7))
plt.semilogy( xhist_gd, label='Gradient Descent (Naive)' )
plt.semilogy( xhist_gdopti, label='Gradient Descent (Opti)' )
k = np.arange(1,maxiter)
plt.semilogy(k,(1-1/kappa)**k,':',label='$(1-\kappa^{-1})^{k}$')
plt.semilogy(k,((kappa-1)/(kappa+1))**k,':',label='$((\kappa-1)/(\kappa+1))^{k}$')
plt.legend( loc='upper right')
plt.grid()
plt.show()

## Chebyshev Acceleration

Using the matrix polynomial: $P^{\text{Grad}}_k(H) = (I-\gamma H)^k$, the gradient descent algorithm can be written as:
\begin{eqnarray*}
x_k-x^* = P^{\text{Grad}}_k(H)(x_0-x^*),
\end{eqnarray*}
and we choose $\gamma$ to ensure $\|I-\gamma H\|_2<1$.

Here, we explore what happens if we can choose a sequence of polynomials in order to speed up the algorithm. The resulting algorithm will be written as:
\begin{eqnarray*}
x_k-x^* = P_k(H)(x_0-x^*).
\end{eqnarray*}
We cannot choose any sequence of polynomials, because we have only access to the gradients of our function evaluated at the points $x_0,\dots, x_{k-1}$ in order to compute $x_{k}$.

### Question 1 (Math)

Show that if for all $k\geq 0$, there exist some sequence of coefficients $\{\alpha_i^{(k)}\}_{i=0\dots k}$ such that
\begin{eqnarray*}
x_k-x_0 = \sum_{i=0}^{k-1} \alpha^{(k-1)}_i \nabla f(x_i)
\end{eqnarray*}
then 
\begin{eqnarray*}
x_k-x^* = P_k(H)(x_0-x^*),
\end{eqnarray*}
with $P_k$ of degree at most $k$ and with $P(0)=1$.

**Solution:**

### Question 2 (Math)

Hence, we consider the worst-case convergence bound by solving:
\begin{eqnarray*}
P^*_k = \arg\min_{P\in\mathcal{P}_k}\max_{\lambda\in[\mu, L]}|P(\lambda)|,
\end{eqnarray*}
where $\mathcal{P}_k$ is the set of polynomials with degree at most $k$ and with $P(0)=1$.

$P^*_k$ are derived from Chebyshev polynomials of the first kind. Chebyshev polynomials of the first kind are defined recursively as follows:
\begin{eqnarray*}
C_0(x) &=& 1,\\
C_1(x) &=&x,\\
C_{k}(x) &=& 2xC_{k-1}(x)-C_{k-2}(x), \text{ for } k\geq 2.
\end{eqnarray*}
We know that they satisfy the minimax property:
\begin{eqnarray*}
\frac{1}{2^{k-1}} C_k = \arg\min_{P\in \mathcal{P}'_k} \max_{x\in [-1,1]} |P(x)|,
\end{eqnarray*}
where $\mathcal{P}'_k$ is the set of monic polynomials with degree less than $k$.
A monic polynomial is a polynomial whose coefficient associated with the highest power is equal to one.

In oder to solve our original problem, we see that we will need to **shift** the interval $[-1,1]$ to $[\mu,L]$ and to **rescale** the polyomial to get $P(0)=1$. This is done thanks to the linear mapping
\begin{eqnarray*}
t^{[\mu,L]}: [\mu, L] &\to& [-1,1]\\
x&\mapsto& \frac{2x-(L+\mu)}{L-\mu}
\end{eqnarray*}
and then
\begin{eqnarray*}
C^{[\mu,L]}_k(x) = \frac{C_k(t^{[\mu,L]}(x))}{C_k(t^{[\mu,L]}(0)}.
\end{eqnarray*}
Using the recursion defining $C_k$, we get:
\begin{eqnarray*}
C^{[\mu,L]}_0(x)&=& 1,\\
C^{[\mu,L]}_1(x)&=&1-\frac{2}{L+\mu}x,\\
C^{[\mu,L]}_k(x) &=& \frac{2\delta_k}{L-\mu}(L+\mu-2x) C^{[\mu,L]}_{k-1}(x) +\left( 1-\frac{2\delta_k(L+\mu)}{L-\mu}\right)C^{[\mu,L]}_{k-2}(x), \text{ for } k\geq 2,
\end{eqnarray*}
with $\delta_1=\frac{L-\mu}{L+\mu}$ and for $k\geq 2$,
\begin{eqnarray*}
\delta_k = \left( 2\frac{L+\mu}{L-\mu}-\delta_{k-1}\right)^{-1}.
\end{eqnarray*}

Show that the resulting algorithm can be written as:
\begin{eqnarray*}
x_k = x_{k-1} -\frac{4\delta_k}{L-\mu}\nabla f(x_{k-1}) + \left( 1-2\delta_k\frac{L+\mu}{L-\mu}\right)(x_{k-2}-x_{k-1}).
\end{eqnarray*}

**solution:**

### Question 3 (Code)

Code the Chebyshev method.

In [ ]:
def Chebyshev_MomemtumGradientDescent(f,grad,L,mu,x0,maxiter=1e3):
    # your code

In [ ]:
x_cmgd, xhist_cmgd = Chebyshev_MomemtumGradientDescent(f,grad,L,mu,x0,maxiter=maxiter)

Compare Chebyshev method to vanilla Gradient descent (the code below should run without any modification).

In [ ]:
plt.figure(figsize=(12,7))
plt.semilogy( xhist_cmgd, label='Chebyshev method' )
plt.semilogy( xhist_gdopti, label='Gradient Descent (Opti)' )
plt.ylim(bottom=1e-13,top=1e2)
k = np.arange(1,maxiter)
plt.semilogy(k,((kappa-1)/(kappa+1))**k,':',label='$((\kappa-1)/(\kappa+1))^{k}$')
plt.semilogy(k,((np.sqrt(kappa)-1)/(np.sqrt(kappa)+1))**k,':',label='$((\sqrt{\kappa}-1)/(\sqrt{\kappa}+1))^{k}$')
plt.legend( loc='upper right')
plt.grid()
plt.show()

### Question 4 (Maths)

What is the limit for $\delta_k$ when $k\to\infty$?

**solution :**

Show that the corresponding recursion when $\delta_k$ is replaced by $\delta_\infty$ in Chebyshev method is:
\begin{eqnarray*}
x_k = x_{k-1} - \frac{4}{(\sqrt{L}+\sqrt{\mu})^2}\nabla f(x_{k-1}) +\frac{(\sqrt{L}-\sqrt{\mu})^2}{(\sqrt{L}+\sqrt{\mu})^2}(x_{k-1}-x_{k-2}),
\end{eqnarray*}
which corresponds to Polyak's heavy-ball method.

### Question 5 (Code)

Code the Polyak's heavy-ball method.

In [ ]:
def MomemtumGradientDescent(f,grad,x0,maxiter=1e3):
    # your code

In [ ]:
x_mgd, xhist_mgd = MomemtumGradientDescent(f,grad,x0,maxiter=maxiter)

Compare all the methods (the code below should run without any modification).

In [ ]:
plt.figure(figsize=(12,7))
plt.semilogy( xhist_cmgd, label='Chebyshev method' )
plt.semilogy( xhist_gdopti, label='Gradient Descent (Opti)' )
plt.semilogy( xhist_mgd, label='Polyak method' )
k = np.arange(1,maxiter)
plt.semilogy(k,((kappa-1)/(kappa+1))**k,':',label='$((\kappa-1)/(\kappa+1))^{k}$')
plt.semilogy(k,((np.sqrt(kappa)-1)/(np.sqrt(kappa)+1))**k,':',label='$((\sqrt{\kappa}-1)/(\sqrt{\kappa}+1))^{k}$')
plt.ylim(bottom=1e-13,top=1e2)
plt.legend( loc='upper right')
plt.grid()
plt.show()